In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import time
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold

In [2]:
train = pd.read_csv('features.csv', header=0, index_col=1)
test = pd.read_csv('features_test.csv', header=0, index_col=1)

train.head()

,match_id,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
start_time,,,,,,,,,,,,,,,,,,,,,
1430198770,0,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1430220345,1,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
1430227081,2,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
1430263531,3,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
1430282290,4,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [3]:
train_y = np.array(train['radiant_win']) # Целевая переменная нахоится в этом столбце

In [4]:
# Фичи, отсутствующие в тестовом датасете
a = list(train.keys())
b = list(test.keys())
future_feature = []
for i in a:
    if b.count(i) == 0:
        future_feature.append(i)

# Удаление этих фич
train = train.drop(future_feature, axis=1)
future_feature

['duration',
 'radiant_win',
 'tower_status_radiant',
 'tower_status_dire',
 'barracks_status_radiant',
 'barracks_status_dire']

In [5]:
# Выделение признаков с пропусками
nans = {}
nan_feature = []
for k in train.keys():
    c = 0
    for i in train[k]:
        if np.isnan(i):
            c += 1
            nans[str(k)] = c
            if nan_feature.count(k) == 0:
                nan_feature.append(k)
        
nans # Признаки, имеющие пропуки : количество пропусков
# first_blood_time - отсутствие значение этого признака может говорить о том,
# что за первы 5 минут игры ни одна из команд не сделала "первую кровь".

# dire_flying_courier_time - если пропущен этот признак, то это может
# значить, что команда dire за первые 5 минут не апнула курьера до "летающего курьера".


{'first_blood_time': 19553,
 'first_blood_team': 19553,
 'first_blood_player1': 19553,
 'first_blood_player2': 43987,
 'radiant_bottle_time': 15691,
 'radiant_courier_time': 692,
 'radiant_flying_courier_time': 27479,
 'radiant_first_ward_time': 1836,
 'dire_bottle_time': 16143,
 'dire_courier_time': 676,
 'dire_flying_courier_time': 26098,
 'dire_first_ward_time': 1826}

In [6]:
# Заполнение пропусков средним по признаку
means = {}
means_test = {}
for k in nan_feature:
    means[str(k)] = np.mean(train[k])
    means_test[str(k)] = np.mean(test[k])

nonans_train = train.fillna(means)
nonans_test = test.fillna(means_test)

In [7]:
train_X = np.array(nonans_train)
test_X = np.array(nonans_test)

In [8]:
# Кросс валидация на 30 деревьях
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier

t1 = int(round(time.time() * 1000))  # millis

kf = KFold(5, shuffle=True)
gb = GradientBoostingClassifier(learning_rate=0.3, n_estimators=30, validation_fraction=0.2, n_iter_no_change=None, tol=0.01)
sc = cross_val_score(gb, train_X, train_y, cv=kf, scoring='roc_auc')
t2 = int(round(time.time() * 1000))  # millis
print("Computation time:  %.1f sec" % ((t2-t1) / 1e3))

Computation time:  104.2 sec


In [64]:
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier

t1 = int(round(time.time() * 1000))  # millis

#roc = make_scorer(roc_auc_score)
#skf = StratifiedKFold(5, shuffle=True)
kf = KFold(5, shuffle=True)
# Кросс валидация на градиентном бустинге
for n in [10, 20, 30]:
    gb = GradientBoostingClassifier(learning_rate=0.3, n_estimators=n, validation_fraction=0.2, n_iter_no_change=None, tol=0.01)
    sc = cross_val_score(gb, train_X, train_y, cv=kf, scoring='roc_auc')
    print(f"cross vall score on {n} estimators : {sc}, - average score: {np.mean(sc)}")
        
t2 = int(round(time.time() * 1000))  # millis
print("Computation time:  %.1f sec" % ((t2-t1) / 1e3))

cross vall score on 10 estimators with lr 0.1: [0.66220941 0.66662966 0.66371979 0.66169126 0.66219963], - average score: 0.6632899494647415
cross vall score on 10 estimators with lr 0.2: [0.67731899 0.67890862 0.68096246 0.67478588 0.68277847], - average score: 0.6789508850491204
cross vall score on 10 estimators with lr 0.3: [0.6748488  0.68093804 0.67945446 0.68596247 0.67802363], - average score: 0.679845478268435
cross vall score on 20 estimators with lr 0.1: [0.68357032 0.68161628 0.68098891 0.68110175 0.68225837], - average score: 0.681907124928069
cross vall score on 20 estimators with lr 0.2: [0.69084557 0.69404433 0.68792893 0.69451775 0.68781859], - average score: 0.6910310327296582
cross vall score on 20 estimators with lr 0.3: [0.6998315  0.6949601  0.69278283 0.69416496 0.68888209], - average score: 0.6941242966979072
cross vall score on 30 estimators with lr 0.1: [0.69267797 0.68835084 0.68364498 0.69159298 0.68921548], - average score: 0.6890964491729922
cross vall scor

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

kf = KFold(5, shuffle=True)
sc = StandardScaler()
train_X = sc.fit_transform(train_X)
# Кросс валидация на логистической регресии
t1 = int(round(time.time() * 1000))  # millis

for c in list(range(1, 30)):
    login = LogisticRegression(C=c, fit_intercept=True, solver='lbfgs', verbose=0, n_jobs=-1)
    sc = cross_val_score(login, train_X, train_y, cv=kf, scoring='roc_auc')
    print(f"cross vall score on {c} regularize : {sc}, - average score: {np.mean(sc)}")

t2 = int(round(time.time() * 1000))  # millis
print("Computation time:  %.1f sec" % ((t2-t1) / 1e3))

cross vall score on 1 regularize : [0.71745563 0.71008875 0.71194941 0.7211611  0.72432102], - average score: 0.7169951823396858
cross vall score on 2 regularize : [0.71918819 0.72048948 0.71448017 0.71946808 0.71101124], - average score: 0.716927432045858
cross vall score on 3 regularize : [0.71915659 0.71862227 0.71337195 0.71793315 0.71539098], - average score: 0.7168949844561516
cross vall score on 4 regularize : [0.72066232 0.7166996  0.71443285 0.71636607 0.71567901], - average score: 0.7167679693082503
cross vall score on 5 regularize : [0.71633325 0.71727044 0.7175833  0.71187388 0.72018389], - average score: 0.7166489520432657
cross vall score on 6 regularize : [0.71753239 0.71444064 0.71698778 0.7166428  0.71795431], - average score: 0.7167115854888871
cross vall score on 7 regularize : [0.71613308 0.71553247 0.71800927 0.7138693  0.72177332], - average score: 0.7170634876861373
cross vall score on 8 regularize : [0.7158945  0.71967005 0.71538471 0.72028569 0.71133495], - ave

In [12]:
categ_features = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 
                  'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
log_train_X = nonans_train.drop(categ_features, axis=1)
log_test_X = nonans_test.drop(categ_features, axis=1)

In [13]:
# Кросс валидация на логистической регресии без категориальных фич
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
log_train_X = sc.fit_transform(log_train_X)
t1 = int(round(time.time() * 1000))  # millis

for c in list(range(1, 30)):
    login = LogisticRegression(C=c, fit_intercept=True, solver='liblinear', verbose=0, n_jobs=-1)
    sc = cross_val_score(login, log_train_X, train_y, cv=kf, scoring='roc_auc')
    print(f"cross vall score on {c} regularize : {sc}, - average score: {np.mean(sc)}")

t2 = int(round(time.time() * 1000))  # millis
print("Computation time:  %.1f sec" % ((t2-t1) / 1e3))

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
c:\users\user\appdata\local\programs\python\python36\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


cross vall score on 1 regularize : [0.71325644 0.72391558 0.71136242 0.72075949 0.71453254], - average score: 0.7167652939686016
cross vall score on 2 regularize : [0.71942139 0.71614515 0.71582143 0.71851317 0.71537494], - average score: 0.7170552153070441
cross vall score on 3 regularize : [0.72634715 0.71983395 0.71124219 0.71646333 0.71085128], - average score: 0.7169475818378974
cross vall score on 4 regularize : [0.7195599  0.72508938 0.70838656 0.71376278 0.71711157], - average score: 0.716782039989379
cross vall score on 5 regularize : [0.71560272 0.71192382 0.71837967 0.72073046 0.71809318], - average score: 0.7169459698425675
cross vall score on 6 regularize : [0.72212634 0.716058   0.7137589  0.71188234 0.71996366], - average score: 0.7167578471162454
cross vall score on 7 regularize : [0.7205449  0.71626934 0.71689334 0.71298233 0.71796634], - average score: 0.7169312480829405
cross vall score on 8 regularize : [0.71587422 0.71362567 0.72467991 0.71547019 0.71439577], - ave

In [15]:
# все герои, участвующие в обучающей выборке
all_heros = []
for feature in categ_features[1:]:
    all_heros = np.concatenate((all_heros, nonans_train[feature]))

heroes_count = np.unique(all_heros)
n = heroes_count.shape[0] # 108 героев

# Добавление мешка слов вместо id героев
wb = {}
ws = {}
for id in heroes_count:
    # Мы используем + 0 для автоматического приведения bool-->int.
    r = [(nonans_train['r%d_hero' % n] == id) + 0 for n in range(1, 6)]
    d = [(nonans_train['d%d_hero' % n] == id) + 0 for n in range(1, 6)]
    
    s = [(nonans_test['r%d_hero' % n] == id) + 0 for n in range(1, 6)]
    k = [(nonans_test['d%d_hero' % n] == id) + 0 for n in range(1, 6)]
    wb['hero%s' % id] = sum(r) - sum(d)
    
    ws['hero%s' % id] = sum(s) - sum(k)
X_pick = nonans_train.assign(**wb)
x_pick = nonans_test.assign(**ws)

log_train_X = X_pick.drop(categ_features, axis=1)
log_test_X = x_pick.drop(categ_features, axis=1)
log_test_X

,match_id,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,hero100.0,hero101.0,hero102.0,hero103.0,hero104.0,hero105.0,hero106.0,hero109.0,hero110.0,hero112.0
start_time,,,,,,,,,,,,,,,,,,,,,
1430287923,6,4,1103,1089,8,0,1,9,3,1183,...,0,0,1,0,0,0,0,0,0,0
1430293357,7,2,556,570,1,0,0,9,4,1194,...,0,0,0,0,0,0,0,0,0,0
1430301774,10,2,751,808,1,0,0,13,2,421,...,0,0,0,0,0,0,0,0,0,1
1430323933,13,3,708,903,1,1,1,11,2,672,...,0,0,0,0,0,0,0,0,-1,0
1430331112,16,4,1259,661,4,0,0,9,5,1703,...,0,0,1,0,0,0,0,0,0,0
1430334264,18,5,2067,1549,15,0,0,8,4,1258,...,0,0,0,0,0,0,0,0,0,0
1430334995,19,4,1862,1487,24,0,1,5,3,894,...,0,-1,0,0,0,0,0,0,0,1
1430349466,24,5,1991,1983,19,2,0,10,4,1498,...,0,0,0,0,0,0,1,0,0,0
1430366484,33,2,517,500,0,0,1,9,4,1585,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Кросс валидация на логистической регресии с мешком слов
t1 = int(round(time.time() * 1000))  # millis

for c in list(range(30, 32)):
    login = LogisticRegression(C=c, fit_intercept=True, solver='liblinear', verbose=0, n_jobs=-1)
    sc = cross_val_score(login, log_train_X, train_y, cv=kf, scoring='roc_auc')
    print(f"cross vall score on {c} regularize : {sc}, - average score: {np.mean(sc)}")

t2 = int(round(time.time() * 1000))  # millis
print("Computation time:  %.1f sec" % ((t2-t1) / 1e3))

cross vall score on 30 regularize : [0.74648851 0.74779891 0.73511007 0.73924465 0.73653736], - average score: 0.7410358990193497
cross vall score on 31 regularize : [0.73716726 0.73507198 0.74671645 0.74162764 0.74382189], - average score: 0.7408810451374724
Computation time:  483.4 sec


In [17]:
from sklearn.metrics import roc_auc_score
# Лучшая модель
login = LogisticRegression(C=31, fit_intercept=True, solver='liblinear', verbose=0, n_jobs=-1)
login.fit(log_train_X, train_y)
print('roc_auc:', roc_auc_score(train_y, login.predict_proba(log_train_X)[:, 1]))

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:1296: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


roc_auc: 0.7367802426564234


In [23]:
preds = login.predict_proba(log_test_X)[:,1]
max_pred = max(preds)
min_pred = min(preds)
print(max_pred)
print(min_pred)

0.9899028465431151
0.00630988295051432


In [24]:
# Предсказание вероятностей победы radiant на тестовой выборке
preds = {'match_id': [], 'radiant_win': []}
pred = login.predict_proba(log_train_X)[:, 1]
for i in range(log_train_X.shape[0]):
    preds['radiant_win'].append(pred[i])
    preds['match_id'].append(i)

rez = DataFrame(preds)
rez.to_csv('rezult.csv')
rez

,match_id,radiant_win
0,0,0.570804
1,1,0.474459
2,2,0.697612
3,3,0.366985
4,4,0.457462
5,5,0.233248
6,6,0.158704
7,7,0.394985
8,8,0.302266
9,9,0.373640
